# Differentiating CVQNN layers with Piquasso and Tensorflow

In Piquasso, one can easily create Continuous-Variable Quantum Neural Networks (CVQNN) circuits using the [Piquasso CVQNN](https://piquasso.readthedocs.io/en/latest/advanced/cvqnn.html) module. In these circuits, the layers are constructed according to [Continuous-variable quantum neural networks](https://arxiv.org/abs/1806.06871).

In [8]:
import piquasso as pq

d = 2  # Number of qumodes

layer_count = 10  # Number of CVQNN layers

# Generating random weights
weights = pq.cvqnn.generate_random_cvqnn_weights(layer_count=layer_count, d=d)

# Creating CVQNN layers as a Piquasso subprogram
cvqnn_layers = pq.cvqnn.create_layers(weights)

Piquasso automatically sets up a subprogram containing the instructions of the desired CVQNN layer. Now we can embed this subprogram in any Piquasso program. Let's choose the input state as a pure displaced state.

In [9]:
# The program definition
with pq.Program() as program:
    pq.Q() | pq.Vacuum()

    for i in range(d):
        pq.Q(i) | pq.Displacement(r=0.1)

    pq.Q() | cvqnn_layers

Now, we need to choose the simulator which executes the instructions. Since a CVQNN layer includes non-linear terms, we definitely need to perform the simulation in Fock space. Since our initial state is pure, we can use [PureFockSimulator](https://piquasso.readthedocs.io/en/latest/simulators/fock.html#piquasso._simulators.fock.pure.simulator.PureFockSimulator).

In [10]:
cutoff = 5

simulator = pq.PureFockSimulator(d, pq.Config(cutoff=cutoff))

final_state = simulator.execute(program).state

After obtaining the state, we can calculate several things, e.g. the expectation value of the position operator on mode 0.

In [11]:
print("Mean position on mode 0:")
final_state.mean_position(mode=0)

Mean position on mode 0:


0.08810452589019563

In order to differentiate quantitities, we need to modify the simulation. In itself, `PureFockSimulator` is unable to perform automatic differentiation. In order to do that, we can use `TensorflowConnector`, which replaces NumPy to TensorFlow under the hood. For a concrete example, let the loss function be
$$
J(w) = || \ket{\psi(w)} - \ket{\psi_*} ||_2,
$$
where $\ket{\psi(w)}$ is the final state of the circuit, and $\ket{\psi_*}$ is some random final state.

In [12]:
import numpy as np
from scipy.special import comb

state_vector_size = comb(d + cutoff - 1, cutoff - 1, exact=True)

psi_star = np.random.rand(state_vector_size) + 1j * np.random.rand(state_vector_size)

psi_star /= np.sum(np.abs(psi_star))

Then, by using [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape), we can differentiate this loss function.

In [13]:
import tensorflow as tf

tf.get_logger().setLevel("ERROR")  # Turns off complex->float casting warnings

simulator = pq.PureFockSimulator(
    d, pq.Config(cutoff=cutoff), connector=pq.TensorflowConnector()
)

w = tf.Variable(weights)
psi_star = tf.Variable(psi_star)

with tf.GradientTape() as tape:
    cvqnn_layers = pq.cvqnn.create_layers(w)

    with pq.Program() as program:
        pq.Q() | pq.Vacuum()

        for i in range(d):
            pq.Q(i) | pq.Displacement(r=0.1)

        pq.Q() | cvqnn_layers

    simulator.execute(program)

    final_state = simulator.execute(program).state

    psi = final_state.state_vector

    loss = tf.math.reduce_sum(tf.math.abs(psi - psi_star))

loss_grad = tape.gradient(loss, w)

print(f"Loss: {loss.numpy()}")
print(f"Loss gradient: {loss_grad.numpy()}")

Loss: 1.9604476480647497
Loss gradient: [[ 2.48370504e-02  6.91253512e-03 -5.17538228e-02  5.91197083e-01
   8.35270137e-01  4.33414809e-02 -1.44455276e-03 -6.10485008e-02
   6.30518617e-01  7.87589392e-01  4.15688561e-04  6.45192857e-03
  -7.05893753e-02 -1.81552492e-02]
 [ 5.51070724e-02  1.14482055e-02 -7.20810177e-02  8.70276885e-01
   6.50958369e-01  4.79129674e-02 -1.05355280e-02 -6.39677085e-02
   7.03435046e-01  7.72832475e-01  6.62571242e-03  1.03948801e-03
  -7.14190194e-02 -1.32908739e-02]
 [ 5.91775056e-02  4.75369308e-03 -6.20956892e-02  8.06572060e-01
   7.06752388e-01  4.33605125e-02 -4.52911124e-04 -6.26476669e-02
   7.98818963e-01  7.21751009e-01  1.01077259e-02 -2.26709094e-03
  -6.75334937e-02 -1.38186903e-02]
 [ 4.01203598e-02  1.99768691e-03 -5.45376280e-02  8.81982242e-01
   6.53387621e-01  5.93868412e-03 -6.30429008e-04 -5.38798294e-02
   9.05422584e-01  7.15949040e-01 -3.64702223e-04 -5.77410545e-05
  -6.82197114e-02  1.74116637e-03]
 [ 1.07959822e-03  1.7305848

However, Piquasso is written in a way that it supports `tf.function` (see [Better performance with tf.function](https://www.tensorflow.org/guide/function)) one can also use `tf.function` for this task. Refactoring everything into a function, we can use the `tf.function` decorator. Note, that we have to avoid side effects in any function decorated with `tf.function`, because side effects are only executed at the tracing step. Therefore, instantiation of `pq.Program` should happen by providing the instructions as constructor arguments, instead of using the `with` keyword.

In [14]:
def calculate_loss(w, psi_star, cutoff):
    d = pq.cvqnn.get_number_of_modes(w.shape[1])

    simulator = pq.PureFockSimulator(
        d,
        pq.Config(cutoff=cutoff),
        connector=pq.TensorflowConnector(decorate_with=tf.function),
    )

    with tf.GradientTape() as tape:
        cvqnn_layers = pq.cvqnn.create_layers(w)

        final_state = simulator.execute_instructions(
            instructions=[pq.Vacuum()] + cvqnn_layers.instructions
        ).state

        psi = final_state.state_vector

        loss = tf.math.reduce_sum(tf.math.abs(psi - psi_star))

    return loss, tape.gradient(loss, w)


improved_calculate_loss = tf.function(calculate_loss)

loss, loss_grad = improved_calculate_loss(w, psi_star, cutoff)

print(f"Loss: {loss.numpy()}")
print(f"Loss gradient: {loss_grad.numpy()}")

Loss: 2.018569979174099
Loss gradient: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.75160857e-01
   6.83322458e-01  1.28450675e-02 -4.00008041e-05 -6.30548348e-03
  -7.49845629e-01 -8.00313533e-01  7.09929231e-05  1.03634563e-02
  -1.18960333e-02  1.04187002e-02]
 [-1.35800236e-03 -5.49585214e-04 -5.68490534e-03  7.68436902e-01
   5.05490527e-01  6.69074565e-03  6.61089567e-04 -5.98396524e-03
  -7.70615878e-01 -7.65752853e-01  2.44147270e-03  2.03468039e-03
  -4.38080826e-03  1.26999161e-02]
 [ 1.33582634e-02 -1.68655402e-03 -1.85593853e-03  7.07517913e-01
   5.44470265e-01  2.49539746e-02  6.42390173e-05 -1.95025626e-03
  -8.25024061e-01 -6.44185279e-01  6.53653384e-03 -7.65527162e-03
   9.10541653e-03  3.40278572e-03]
 [ 4.26398634e-02 -1.80787941e-03  6.39415698e-03  7.10539771e-01
   5.16317806e-01  4.19027819e-02  3.11151157e-04  6.34437434e-03
  -7.11899830e-01 -5.86043987e-01 -1.78246076e-03 -3.32723452e-04
   1.04650909e-02  4.07298677e-03]
 [ 4.29666003e-02 -5.13803858

The first run is called the tracing step, and it takes some time, because Tensorflow captures a [tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph) here. The size of the graph can be decreased by passing the `decorate_with=tf.function` argument to `pq.TensorflowConnector`, which also decreases the execution time of the tracing step. After the first run, a significant speedup is observed. We can also compare the runtimes of the compiled and non-compiled function.

In [15]:
import time
import numpy as np

regular_runtimes = []
improved_runtimes = []

for i in range(10):
    w = tf.Variable(pq.cvqnn.generate_random_cvqnn_weights(layer_count, d))

    start_time = time.time()
    calculate_loss(w, psi_star, cutoff)
    end_time = time.time()

    regular_runtimes.append(end_time - start_time)

    start_time = time.time()
    improved_calculate_loss(w, psi_star, cutoff)
    end_time = time.time()

    improved_runtimes.append(end_time - start_time)

print(f"Regular: {np.mean(regular_runtimes)} s (+/- {np.std(regular_runtimes)} s).")
print(f"Improved: {np.mean(improved_runtimes)} s (+/- {np.std(improved_runtimes)} s).")

Regular: 2.4323306560516356 s (+/- 0.25203824522512575 s).
Improved: 0.011875104904174805 s (+/- 0.0002783859661023313 s).


But that is not everything yet! One can also create a similar function with the `jit_compile=True` flag, since every operation in Piquasso can be JIT-compiled using XLA through `tf.function`.

In [16]:
jit_compiled_calculate_loss = tf.function(jit_compile=True)(calculate_loss)

jit_compiled_calculate_loss(w, psi_star, cutoff)

2024-03-02 09:40:10.970536: I external/local_xla/xla/service/service.cc:168] XLA service 0x23ff2f70 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-03-02 09:40:10.970561: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-03-02 09:40:14.366437: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-02 09:42:47.120088: E external/local_xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module a_inference_calculate_loss_184206__XlaMustCompile_true_config_proto_3175580994766145631_executor_type_11160318154034397263_.64111] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2024-03-02 09:45:40.915775: E external/local_xla/xla/service/slow_operation

(<tf.Tensor: shape=(), dtype=float64, numpy=1.9352979440546>,
 <tf.Tensor: shape=(10, 14), dtype=float64, numpy=
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          2.82974314e-01,  1.00761968e+00, -4.86587480e-04,
          2.35804220e-04, -1.15113732e-04, -4.06551101e-01,
         -1.11506339e+00, -3.28812640e-04,  7.26509004e-03,
         -5.50407493e-04,  1.95176250e-02],
        [-1.27337958e-02, -4.32230679e-04, -1.16956930e-05,
          2.97284980e-01,  9.58032369e-01,  7.80146796e-03,
         -1.15800353e-03, -5.29595553e-04, -6.03508410e-01,
         -1.00960026e+00, -6.00128740e-05, -5.43138082e-03,
          1.13088525e-03, -6.98910091e-03],
        [ 1.50953348e-02,  1.06793890e-04, -6.96402317e-04,
          2.78072391e-01,  8.89425807e-01,  1.65284781e-02,
         -3.26193983e-04, -7.32702009e-04, -4.35469885e-01,
         -1.08220389e+00, -2.19933132e-03, -2.47865138e-03,
         -2.93182299e-03, -1.68765726e-02],
        [ 1.09036764e-02, -1.399439

Compiling the same function takes significantly more time, but the compilation step results in an extra order of magnitude runtime improvement:

In [17]:
jit_compiled_runtimes = []

for i in range(10):
    w = tf.Variable(pq.cvqnn.generate_random_cvqnn_weights(layer_count, d))

    start_time = time.time()
    jit_compiled_calculate_loss(w, psi_star, cutoff)
    end_time = time.time()

    jit_compiled_runtimes.append(end_time - start_time)

print(f"Regular:\t{np.mean(regular_runtimes)} s (+/- {np.std(regular_runtimes)} s).")
print(f"Improved:\t{np.mean(improved_runtimes)} s (+/- {np.std(improved_runtimes)} s).")
print(
    f"JIT compiled:\t{np.mean(jit_compiled_runtimes)} s "
    f"(+/- {np.std(jit_compiled_runtimes)} s)."
)

Regular:	2.4323306560516356 s (+/- 0.25203824522512575 s).
Improved:	0.011875104904174805 s (+/- 0.0002783859661023313 s).
JIT compiled:	0.0013802051544189453 s (+/- 0.00014840260989162237 s).


We are able to use this function for. e.g., quantum state learning. Consider the state
$$
\ket{\psi_*} = \frac{1}{\sqrt{2}} \left ( \ket{03} + \ket{30} \right ),
$$
which is an example of a so-called NOON state. We can produce this using Piquasso:

In [18]:
with pq.Program() as target_state_preparation:
    pq.Q(all) | pq.StateVector([0, 3]) / np.sqrt(2)
    pq.Q(all) | pq.StateVector([3, 0]) / np.sqrt(2)


target_state = simulator.execute(target_state_preparation).state

target_state_vector = target_state.state_vector

psi_star = tf.Variable(target_state_vector)

Now we can demonstrate the speed of the JIT-compiled calculation by creating a simple optimization algorithm as follows:

In [19]:
learning_rate = 0.01
iterations = 10000

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

w = tf.Variable(pq.cvqnn.generate_random_cvqnn_weights(layer_count, d))


for i in range(iterations):
    loss, loss_grad = jit_compiled_calculate_loss(w, psi_star, cutoff)

    optimizer.apply_gradients(zip([loss_grad], [w]))

    if (i + 1) % (iterations // 20) == 0:
        print(f"{i + 1}:\t\t", loss.numpy())

print("Final loss:\t", loss.numpy())

500:		 0.8607086996549679
1000:		 0.8473090910243798
1500:		 0.5849377384183669
2000:		 0.40879991260231896
2500:		 0.29813437623431027
3000:		 0.2761913995731654
3500:		 0.26621822690380204
4000:		 0.23435492471006458
4500:		 0.23874428115482763
5000:		 0.2205772331178741
5500:		 0.2416911247101965
6000:		 0.2146732713315681
6500:		 0.1993502777279203
7000:		 0.22721179422812682
7500:		 0.21320290731217184
8000:		 0.20555180951647123
8500:		 0.21018212032348688
9000:		 0.20439647783148554
9500:		 0.19759860554592923
10000:		 0.18922056395791387
Final loss:	 0.18922056395791387


We can use the final weights to calculate the final state, and calculate its fidelity with the target state.

In [20]:
program = pq.cvqnn.create_program(w)

final_state = simulator.execute(program).state

print("Final state fidelity: ", final_state.fidelity(target_state))

Final state fidelity:  0.9994946525680765
